# Experiment: _Node weights_

In [ ]:
from notebook_prelude import *

In [ ]:
EXPERIMENT_NAME = 'experiment_node_weights'
df = results_helper.get_experiments_by_names([EXPERIMENT_NAME])

In [ ]:
df.columns

In [ ]:
for x in ['graph__fast_wl__node_weight_iteration_weight_function', 'graph__fast_wl__node_weight_iteration_weight_function']:
    df[x + '_'] = df[x].apply(lambda x: x.__name__ if callable(x) else '-')

df['graph__fast_wl__node_weight_iteration_weight_function_']  =df.graph__fast_wl__node_weight_iteration_weight_function.apply(lambda x: x.__name__ if callable(x) else '-')

In [ ]:
df_ = df.groupby(['dataset', 'graph__fast_wl__node_weight_function']).mean_test_f1_macro.max().to_frame().unstack()
df_['difference_non_vs_weight'] = df_['mean_test_f1_macro', 'none'] - df_['mean_test_f1_macro', 'nxgraph_degrees_metric']
#df_['mean_test_f1_macro', 'none']
df_